In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas_profiling 

# 전처리
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
log_data = pd.read_csv('/content/gdrive/MyDrive/dataset/log_data.csv')
spec_data = pd.read_csv('/content/gdrive/MyDrive/dataset/user_spec.csv')
loan_data = pd.read_csv('/content/gdrive/MyDrive/dataset/loan_result.csv')
relog_data = pd.read_csv('/content/gdrive/MyDrive/dataset/relog.csv')

In [4]:
relog_data

,Unnamed: 0,user_id,timestamp,event
0,11709372,1,2022-05-03 14:52:28,11-11-2-8-11
1,10428909,7,2022-05-22 16:39:49,11
2,9627339,9,2022-05-21 23:37:58,11-11-11
3,9505105,11,2022-03-24 10:53:59,1-11-8-11-9-6-5-4-7-11-8-8-1-11-8-5-4-7-11-8-8...
4,1447554,12,2022-03-14 01:13:11,1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-7-1-2-6-5-4-7-7-...
...,...,...,...,...
584631,2012632,879693,2022-05-13 11:29:49,6-5-4-7-1-7-6-1-8-11-6-7-1-6-5-4-6-5-4-6-7-7-7...
584632,14380169,879694,2022-03-31 20:07:23,6-5-4-7-7-7
584633,6068504,879695,2022-05-27 12:48:32,6-5-4-7
584634,7298665,879696,2022-03-14 05:35:34,2-8-11-11-8-11-1-2-5-4-7


- 7-7-7, 11-11-11과 같이 반복되는 데이터는 무엇일까...? 앱을 실행시켜 놓은 후 다른 앱을 실행하거나, 메신저등의 행동을 하고 온사람들...?
- 어떻게 로그인 부터 안하고 다른 행동들을 먼저 시작하였는가

1. 핀다 사용해보면서 순서에 따라 행동을 정의해야 할 필요성
2. 파생변수 :
- 하루에 몇번 앱에 접속하였는지
- 총 몇번 앱에 접속하였는지
- 각 행동으로 넘어갈때 소요시간 (행동의 time_stamp - 그 다음 행동의 time_stamp) 그리고 이것들의 평균을 구한다음 가장 오래걸리는 행동 파악
- 신규 사용자: 사용경험이 1회인 사용자
- 재방문 사용자: 사용경험이 2회 이상인 사용자


- use_loanmanage : 대출관리서비스 이용 : is applied 가 1인지 확인하기 나중에
- 신규 이용자와 기존 이용자 구분하기
- 한도조회 서비스 이용 고객 : 1. sign up 2.open App 3.Login 4.ViewLoanApplyIntro 5. Start Loan Apply 6.CompleteIDCertification 7. EndLoanApply
- 대출을 이미 했던 고객 : UseLoanManage
- 나머지 자잘한 핀다 서비스 이용 고객들 : UsePrepayCalc, UseDSRCalc, GetCreditinfo

In [6]:
log_data=log_data.sort_values(by=["user_id","timestamp","event"], ascending=[True,True,True])

In [7]:
log_data

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
11709372,1,GetCreditInfo,2022-05-03 14:52:28,android,464,2022-05-03
11709374,1,GetCreditInfo,2022-05-03 14:52:35,android,464,2022-05-03
2451693,1,Login,2022-06-16 23:58:41,Android,3.12.1,2022-06-16
2451691,1,UseLoanManage,2022-06-16 23:58:41,Android,3.12.1,2022-06-16
7071607,1,GetCreditInfo,2022-06-16 23:58:42,android,464,2022-06-16
...,...,...,...,...,...,...
7298671,879696,CompleteIDCertification,2022-03-14 05:38:16,Android,3.8.2,2022-03-14
7298672,879696,EndLoanApply,2022-03-14 05:42:14,Android,3.8.2,2022-03-14
8248077,879698,OpenApp,2022-05-24 22:33:24,iOS,NaN,2022-05-24
8248080,879698,StartLoanApply,2022-05-24 22:33:32,iOS,NaN,2022-05-24


In [61]:
log_data

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
11709372,1,GetCreditInfo,2022-05-03 14:52:28,android,464,2022-05-03
11709374,1,GetCreditInfo,2022-05-03 14:52:35,android,464,2022-05-03
2451693,1,Login,2022-06-16 23:58:41,Android,3.12.1,2022-06-16
2451691,1,UseLoanManage,2022-06-16 23:58:41,Android,3.12.1,2022-06-16
7071607,1,GetCreditInfo,2022-06-16 23:58:42,android,464,2022-06-16
...,...,...,...,...,...,...
7298671,879696,CompleteIDCertification,2022-03-14 05:38:16,Android,3.8.2,2022-03-14
7298672,879696,EndLoanApply,2022-03-14 05:42:14,Android,3.8.2,2022-03-14
8248077,879698,OpenApp,2022-05-24 22:33:24,iOS,NaN,2022-05-24
8248080,879698,StartLoanApply,2022-05-24 22:33:32,iOS,NaN,2022-05-24


각 유저 아이디당 하루에 행동(event)을 얼마나 했는지

In [72]:
grouped = log_data.groupby(['user_id', 'date_cd'])['user_id'].count()
grouped

user_id  date_cd   
1        2022-05-03     2
         2022-06-16     3
7        2022-05-22     1
9        2022-05-21     3
11       2022-03-24    12
                       ..
879693   2022-06-29     1
879694   2022-03-31     6
879695   2022-05-27     4
879696   2022-03-14    11
879698   2022-05-24     3
Name: user_id, Length: 2728153, dtype: int64

In [73]:
df_grouped = pd.DataFrame(grouped)
df_grouped

user_id
user_id date_cd            
1       2022-05-03        2
        2022-06-16        3
7       2022-05-22        1
9       2022-05-21        3
11      2022-03-24       12
...                     ...
879693  2022-06-29        1
879694  2022-03-31        6
879695  2022-05-27        4
879696  2022-03-14       11
879698  2022-05-24        3

[2728153 rows x 1 columns]

In [74]:
df_grouped.rename(columns = {'user_id' : 'user_id_cnt'}, inplace = True)
df_grouped

user_id_cnt
user_id date_cd                
1       2022-05-03            2
        2022-06-16            3
7       2022-05-22            1
9       2022-05-21            3
11      2022-03-24           12
...                         ...
879693  2022-06-29            1
879694  2022-03-31            6
879695  2022-05-27            4
879696  2022-03-14           11
879698  2022-05-24            3

[2728153 rows x 1 columns]

In [75]:
df_grouped_new = df_grouped.reset_index('date_cd')
df_grouped_new

,date_cd,user_id_cnt
user_id,,
1,2022-05-03,2
1,2022-06-16,3
7,2022-05-22,1
9,2022-05-21,3
11,2022-03-24,12
...,...,...
879693,2022-06-29,1
879694,2022-03-31,6
879695,2022-05-27,4


In [76]:
df_grouped_new = df_grouped_new.reset_index('user_id')
df_grouped_new

,user_id,date_cd,user_id_cnt
0,1,2022-05-03,2
1,1,2022-06-16,3
2,7,2022-05-22,1
3,9,2022-05-21,3
4,11,2022-03-24,12
...,...,...,...
2728148,879693,2022-06-29,1
2728149,879694,2022-03-31,6
2728150,879695,2022-05-27,4
2728151,879696,2022-03-14,11


In [77]:
df_grouped_new['user_id_cnt'].groupby(df_grouped_new['user_id']).sum()

user_id
1           5
7           1
9           3
11         43
12        120
         ... 
879693     61
879694      6
879695      4
879696     11
879698      3
Name: user_id_cnt, Length: 584636, dtype: int64

유저가 기간동안 총 몇일 접속했는지

In [78]:
grouped = log_data.groupby(['user_id', 'date_cd'])['user_id'].count()
grouped

user_id  date_cd   
1        2022-05-03     2
         2022-06-16     3
7        2022-05-22     1
9        2022-05-21     3
11       2022-03-24    12
                       ..
879693   2022-06-29     1
879694   2022-03-31     6
879695   2022-05-27     4
879696   2022-03-14    11
879698   2022-05-24     3
Name: user_id, Length: 2728153, dtype: int64

In [79]:
df_grouped = pd.DataFrame(grouped)
df_grouped

user_id
user_id date_cd            
1       2022-05-03        2
        2022-06-16        3
7       2022-05-22        1
9       2022-05-21        3
11      2022-03-24       12
...                     ...
879693  2022-06-29        1
879694  2022-03-31        6
879695  2022-05-27        4
879696  2022-03-14       11
879698  2022-05-24        3

[2728153 rows x 1 columns]

In [80]:
df_grouped.rename(columns = {'user_id' : 'user_id_cnt'}, inplace = True)
df_grouped

user_id_cnt
user_id date_cd                
1       2022-05-03            2
        2022-06-16            3
7       2022-05-22            1
9       2022-05-21            3
11      2022-03-24           12
...                         ...
879693  2022-06-29            1
879694  2022-03-31            6
879695  2022-05-27            4
879696  2022-03-14           11
879698  2022-05-24            3

[2728153 rows x 1 columns]

In [81]:
df_grouped_new = df_grouped.reset_index('date_cd')
df_grouped_new

,date_cd,user_id_cnt
user_id,,
1,2022-05-03,2
1,2022-06-16,3
7,2022-05-22,1
9,2022-05-21,3
11,2022-03-24,12
...,...,...
879693,2022-06-29,1
879694,2022-03-31,6
879695,2022-05-27,4


In [82]:
df_grouped_new = df_grouped_new.reset_index('user_id')
df_grouped_new

,user_id,date_cd,user_id_cnt
0,1,2022-05-03,2
1,1,2022-06-16,3
2,7,2022-05-22,1
3,9,2022-05-21,3
4,11,2022-03-24,12
...,...,...,...
2728148,879693,2022-06-29,1
2728149,879694,2022-03-31,6
2728150,879695,2022-05-27,4
2728151,879696,2022-03-14,11


In [83]:
df_grouped_new['user_id_cnt'].groupby(df_grouped_new['user_id']).count()

user_id
1          2
7          1
9          1
11         4
12        14
          ..
879693     8
879694     1
879695     1
879696     1
879698     1
Name: user_id_cnt, Length: 584636, dtype: int64

=========================

In [8]:
grouped = log_data.groupby(['user_id', 'event'])['timestamp'].min()
grouped

user_id  event             
1        GetCreditInfo         2022-05-03 14:52:28
         Login                 2022-06-16 23:58:41
         UseLoanManage         2022-06-16 23:58:41
7        GetCreditInfo         2022-05-22 16:39:49
9        GetCreditInfo         2022-05-21 23:37:58
                                      ...         
879696   UseLoanManage         2022-03-14 05:35:41
         ViewLoanApplyIntro    2022-03-14 05:37:55
879698   OpenApp               2022-05-24 22:33:24
         StartLoanApply        2022-05-24 22:33:32
         ViewLoanApplyIntro    2022-05-24 22:33:32
Name: timestamp, Length: 2860127, dtype: object

In [9]:
pd.DataFrame(grouped)

timestamp
user_id event                                  
1       GetCreditInfo       2022-05-03 14:52:28
        Login               2022-06-16 23:58:41
        UseLoanManage       2022-06-16 23:58:41
7       GetCreditInfo       2022-05-22 16:39:49
9       GetCreditInfo       2022-05-21 23:37:58
...                                         ...
879696  UseLoanManage       2022-03-14 05:35:41
        ViewLoanApplyIntro  2022-03-14 05:37:55
879698  OpenApp             2022-05-24 22:33:24
        StartLoanApply      2022-05-24 22:33:32
        ViewLoanApplyIntro  2022-05-24 22:33:32

[2860127 rows x 1 columns]

In [10]:
funnel_steps = pd.DataFrame(data = {'step_no':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]},index = ['SignUp','OpenApp','Login','ViewLoanApplyIntro', 'StartLoanApply', 'CompleteIDCertification', 'EndLoanApply', 'UseLoanManage','UsePrepayCalc', 'UseDSRCalc','GetCreditInfo'])
funnel_steps

,step_no
SignUp,1
OpenApp,2
Login,3
ViewLoanApplyIntro,4
StartLoanApply,5
CompleteIDCertification,6
EndLoanApply,7
UseLoanManage,8
UsePrepayCalc,9
UseDSRCalc,10


In [11]:
grouped = pd.DataFrame(grouped).merge(funnel_steps, left_on='event', right_index = True)
grouped

,,timestamp,step_no
user_id,event,,
1,GetCreditInfo,2022-05-03 14:52:28,11
7,GetCreditInfo,2022-05-22 16:39:49,11
9,GetCreditInfo,2022-05-21 23:37:58,11
11,GetCreditInfo,2022-03-24 10:54:07,11
12,GetCreditInfo,2022-05-01 18:30:42,11
...,...,...,...
878796,UseDSRCalc,2022-04-05 21:48:33,10
878870,UseDSRCalc,2022-04-22 18:54:29,10
879287,UseDSRCalc,2022-04-05 17:30:02,10


In [12]:
funnel = grouped.reset_index().pivot(index = 'user_id', columns = 'step_no', values = 'timestamp')
funnel

step_no,1,2,3,4,5,6,7,8,9,10,11
user_id,,,,,,,,,,,
1,NaN,NaN,2022-06-16 23:58:41,NaN,NaN,NaN,NaN,2022-06-16 23:58:41,NaN,NaN,2022-05-03 14:52:28
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-22 16:39:49
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-21 23:37:58
11,NaN,2022-03-24 10:53:59,NaN,2022-03-24 10:55:43,2022-03-24 10:55:43,2022-03-24 10:56:26,2022-03-24 10:59:46,2022-03-24 10:54:08,2022-03-24 10:54:36,NaN,2022-03-24 10:54:07
12,NaN,2022-03-14 01:13:11,2022-03-14 01:13:15,2022-03-14 01:13:22,2022-03-14 01:13:21,2022-03-14 01:13:46,2022-03-14 01:16:26,2022-05-01 18:30:48,NaN,NaN,2022-05-01 18:30:42
...,...,...,...,...,...,...,...,...,...,...,...
879693,NaN,2022-05-13 11:40:29,NaN,2022-05-13 11:29:49,2022-05-13 11:29:49,2022-05-13 11:30:06,2022-05-13 11:34:29,2022-05-13 13:24:32,NaN,NaN,2022-05-13 13:24:33
879694,NaN,NaN,NaN,2022-03-31 20:07:23,2022-03-31 20:07:23,2022-03-31 20:07:42,2022-03-31 20:18:18,NaN,NaN,NaN,NaN
879695,NaN,NaN,NaN,2022-05-27 12:48:32,2022-05-27 12:48:32,2022-05-27 12:48:51,2022-05-27 12:51:23,NaN,NaN,NaN,NaN


In [13]:
funnel.columns  = funnel_steps.index
funnel

,SignUp,OpenApp,Login,ViewLoanApplyIntro,StartLoanApply,CompleteIDCertification,EndLoanApply,UseLoanManage,UsePrepayCalc,UseDSRCalc,GetCreditInfo
user_id,,,,,,,,,,,
1,NaN,NaN,2022-06-16 23:58:41,NaN,NaN,NaN,NaN,2022-06-16 23:58:41,NaN,NaN,2022-05-03 14:52:28
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-22 16:39:49
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-21 23:37:58
11,NaN,2022-03-24 10:53:59,NaN,2022-03-24 10:55:43,2022-03-24 10:55:43,2022-03-24 10:56:26,2022-03-24 10:59:46,2022-03-24 10:54:08,2022-03-24 10:54:36,NaN,2022-03-24 10:54:07
12,NaN,2022-03-14 01:13:11,2022-03-14 01:13:15,2022-03-14 01:13:22,2022-03-14 01:13:21,2022-03-14 01:13:46,2022-03-14 01:16:26,2022-05-01 18:30:48,NaN,NaN,2022-05-01 18:30:42
...,...,...,...,...,...,...,...,...,...,...,...
879693,NaN,2022-05-13 11:40:29,NaN,2022-05-13 11:29:49,2022-05-13 11:29:49,2022-05-13 11:30:06,2022-05-13 11:34:29,2022-05-13 13:24:32,NaN,NaN,2022-05-13 13:24:33
879694,NaN,NaN,NaN,2022-03-31 20:07:23,2022-03-31 20:07:23,2022-03-31 20:07:42,2022-03-31 20:18:18,NaN,NaN,NaN,NaN
879695,NaN,NaN,NaN,2022-05-27 12:48:32,2022-05-27 12:48:32,2022-05-27 12:48:51,2022-05-27 12:51:23,NaN,NaN,NaN,NaN


In [18]:
funnel_new=funnel[funnel.SignUp.isnull()]
funnel_new

,SignUp,OpenApp,Login,ViewLoanApplyIntro,StartLoanApply,CompleteIDCertification,EndLoanApply,UseLoanManage,UsePrepayCalc,UseDSRCalc,GetCreditInfo
user_id,,,,,,,,,,,
1,NaN,NaN,2022-06-16 23:58:41,NaN,NaN,NaN,NaN,2022-06-16 23:58:41,NaN,NaN,2022-05-03 14:52:28
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-22 16:39:49
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-21 23:37:58
11,NaN,2022-03-24 10:53:59,NaN,2022-03-24 10:55:43,2022-03-24 10:55:43,2022-03-24 10:56:26,2022-03-24 10:59:46,2022-03-24 10:54:08,2022-03-24 10:54:36,NaN,2022-03-24 10:54:07
12,NaN,2022-03-14 01:13:11,2022-03-14 01:13:15,2022-03-14 01:13:22,2022-03-14 01:13:21,2022-03-14 01:13:46,2022-03-14 01:16:26,2022-05-01 18:30:48,NaN,NaN,2022-05-01 18:30:42
...,...,...,...,...,...,...,...,...,...,...,...
879693,NaN,2022-05-13 11:40:29,NaN,2022-05-13 11:29:49,2022-05-13 11:29:49,2022-05-13 11:30:06,2022-05-13 11:34:29,2022-05-13 13:24:32,NaN,NaN,2022-05-13 13:24:33
879694,NaN,NaN,NaN,2022-03-31 20:07:23,2022-03-31 20:07:23,2022-03-31 20:07:42,2022-03-31 20:18:18,NaN,NaN,NaN,NaN
879695,NaN,NaN,NaN,2022-05-27 12:48:32,2022-05-27 12:48:32,2022-05-27 12:48:51,2022-05-27 12:51:23,NaN,NaN,NaN,NaN


In [19]:
funnel['SignUp'].value_counts()

2022-06-26 12:16:37    2
2022-05-21 14:17:52    2
2022-04-22 14:23:50    2
2022-06-29 16:02:28    2
2022-06-22 15:22:15    2
                      ..
2022-05-09 11:53:18    1
2022-06-26 13:08:46    1
2022-06-13 15:26:49    1
2022-03-06 18:47:02    1
2022-06-10 15:57:18    1
Name: SignUp, Length: 28152, dtype: int64

In [21]:
funnel_origin=funnel[funnel.SignUp.notnull()]
funnel_origin

,SignUp,OpenApp,Login,ViewLoanApplyIntro,StartLoanApply,CompleteIDCertification,EndLoanApply,UseLoanManage,UsePrepayCalc,UseDSRCalc,GetCreditInfo
user_id,,,,,,,,,,,
30,2022-03-30 12:13:32,2022-03-27 13:17:48,2022-03-29 21:33:14,2022-03-30 12:12:42,2022-03-30 12:12:41,2022-03-30 12:13:57,2022-03-30 12:16:36,2022-03-30 12:12:31,NaN,NaN,2022-03-30 12:17:08
34,2022-05-13 15:32:03,2022-03-17 15:52:40,2022-03-17 15:52:45,2022-05-10 15:19:26,2022-05-10 15:19:26,2022-05-10 15:20:14,2022-05-10 15:22:30,2022-03-17 15:52:55,NaN,NaN,2022-03-17 15:52:55
44,2022-06-28 11:47:08,2022-06-28 11:44:54,NaN,NaN,NaN,2022-06-28 11:46:59,NaN,2022-06-27 14:53:24,NaN,NaN,2022-06-27 14:53:25
51,2022-05-07 16:53:08,2022-05-03 11:36:31,2022-05-02 11:17:46,2022-05-02 11:18:09,2022-05-02 11:18:08,2022-05-02 11:19:08,2022-05-02 11:24:27,2022-05-02 11:17:53,NaN,NaN,2022-04-29 07:14:23
52,2022-04-14 12:16:00,2022-03-05 18:13:19,2022-03-05 18:13:28,2022-03-10 18:45:17,2022-03-10 18:45:16,2022-03-10 18:45:48,2022-03-10 18:49:14,2022-03-05 18:13:29,NaN,NaN,2022-03-05 18:13:29
...,...,...,...,...,...,...,...,...,...,...,...
879582,2022-03-05 01:05:27,2022-03-03 01:37:39,2022-03-03 01:38:11,2022-03-03 01:38:21,2022-03-03 01:38:21,2022-03-03 01:38:50,2022-03-03 01:39:50,2022-03-03 01:39:55,NaN,NaN,2022-03-03 01:39:52
879591,2022-03-23 21:18:53,2022-03-23 21:04:48,NaN,2022-03-23 20:11:11,NaN,2022-03-23 20:12:27,2022-03-23 20:21:28,NaN,NaN,NaN,2022-03-23 20:10:12
879629,2022-04-12 19:22:58,2022-03-07 09:20:27,2022-03-07 09:20:34,2022-03-12 12:39:28,NaN,2022-03-12 12:40:08,NaN,2022-03-07 09:20:35,NaN,NaN,2022-03-07 09:20:35


In [22]:
log_data

ValueError: ignored